In [2]:
import PIL
import math
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import csv
import pathlib
from os import listdir
from os.path import isfile, join

In [3]:
#image1 = Image.new(mode='L', size = (64,64), color=255)


#draw = ImageDraw.Draw(image1)
#font = ImageFont.truetype("meiryo.ttc", 50)
#font = ImageFont.truetype("yumin.ttf", 50)
#font = ImageFont.truetype("YuGothM.ttc", 50)
#font = ImageFont.truetype("msmincho.ttc", 50)

#draw.text((0, 0), ,(0))
#draw.text((5, 5),u"魔",(0), font=font)
#image1.show()

In [4]:
def createImage(pixelSize=64,symbol=u"魔",key=-1):
    image1 = Image.new(mode='L', size = (pixelSize,pixelSize), color=255)
    draw = ImageDraw.Draw(image1)
    
    
    fontSize = math.ceil(pixelSize*0.78)
    fontOffsetX = math.floor(pixelSize*0.22/2)
    fontOffsetY = math.floor(pixelSize*0.22/4)
    fonts = [
        ImageFont.truetype("meiryo.ttc", fontSize),
        ImageFont.truetype("yumin.ttf", fontSize),
        ImageFont.truetype("YuGothM.ttc", fontSize),
        ImageFont.truetype("msmincho.ttc", fontSize)
    ]
    
    for font in fonts:
        draw.text((fontOffsetX, fontOffsetY),symbol,(0), font=font)
        filename = u"./Training_set/"+str(key) +"_"+ symbol +"_"+ font.getname()[0] +"_"+ font.getname()[1]
        image1.save(filename+".jpg")
        
        draw.rectangle((0,0,pixelSize,pixelSize), 255)
    
#createImage(pixelSize=40,symbole=u"魔",key=0)

def sortValue(item):
    return item[1]

num2jap_dict = {}
jap2num_dict = {}
with open('memrise_rtk_kanji.csv', encoding='utf-8', mode='r') as infile:
    reader = csv.reader(infile)
    
    jap2num_dict = {rows[0]:int(rows[1]) for rows in reader}
    jap_list = list(jap2num_dict.items())
    jap_list.sort(key=sortValue)
    
    for index in range(len(jap_list)):
        symbol = jap_list[index][0]
        key = index+1
        createImage(pixelSize=40,symbol=symbol,key=key)
        #print(symbol + " " + str(key))
   
        
mypath = pathlib.Path().resolve()
picturesNames = [f for f in listdir(mypath) if isfile(join(mypath, f))]